In [1]:
import os

os.chdir("/home/tales/dev/master/mdc_analysis/")
print("working dir", os.getcwd())

import pandas as pd
from pyproj import Proj, transform
import geopy.distance as gp

import matplotlib.pyplot as plt
from src.dao import csv_dao
from src.plot import plot
from src.utils import geo
from src.utils.others import partitions
from src.poi_grabber import google_places

from bokeh.io import output_notebook, show
output_notebook()

pd.set_option('display.float_format', lambda x: '%.4f' % x)

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

working dir /home/tales/dev/master/mdc_analysis


Loading BokehJS ...

## POIs summary 

In [2]:
pois = google_places.load_all_google_places_data()
valid_pois = google_places.load_all_google_places_data(valid_pois=True)

pois["lat_4326"] = pois["latitude"]
pois["lon_4326"] = pois["longitude"]

valid_pois["lat_4326"] = valid_pois["latitude"]
valid_pois["lon_4326"] = valid_pois["longitude"]

print("{} pois".format(len(pois)))
print("{} valid pois".format(len(valid_pois)))
valid_pois.sample(6)

131605 pois
76743 valid pois


,geometry,icon,id,name,opening_hours,photos,place_id,plus_code,price_level,rating,reference,scope,types,user_ratings_total,vicinity,latitude,longitude,lat_4326,lon_4326
124446,"{'location': {'lat': 46.11363, 'lng': 7.496130...",https://maps.gstatic.com/mapfiles/place_api/ic...,b8b85f148a3bb4e571b24fc9880f45e5e392b6dc,Evolène,NaN,"[{'height': 2160, 'html_attributions': ['<a hr...",ChIJC0L8Op0oj0cROh2hs6tk3Ao,NaN,nan,nan,ChIJC0L8Op0oj0cROh2hs6tk3Ao,GOOGLE,"[locality, political]",nan,Evolène,46.1136,7.4961,46.1136,7.4961
99019,"{'location': {'lat': 47.411537, 'lng': 8.52536...",https://maps.gstatic.com/mapfiles/place_api/ic...,9c9138668f76324a075d556decb80d4fec2769ad,Neuaffoltern,NaN,"[{'height': 2322, 'html_attributions': ['<a hr...",ChIJA8nBDPQKkEcRaCH6At6ILbg,"{'compound_code': 'CG6G+J4 Zürich, Switzerland...",nan,nan,ChIJA8nBDPQKkEcRaCH6At6ILbg,GOOGLE,"[bus_station, transit_station, point_of_intere...",nan,Switzerland,47.4115,8.5254,47.4115,8.5254
75867,"{'location': {'lat': 46.686115, 'lng': 7.85642...",https://maps.gstatic.com/mapfiles/place_api/ic...,0c2c7598bfbb9892624a8e59c65edaf2f15a5421,Hotel Metropol,{'open_now': True},"[{'height': 4748, 'html_attributions': ['<a hr...",ChIJmXacLZqkj0cRw7jhIozyjkE,"{'compound_code': 'MVP4+CH Interlaken, Switzer...",nan,4.1000,ChIJmXacLZqkj0cRw7jhIozyjkE,GOOGLE,"[restaurant, food, lodging, point_of_interest,...",483.0000,"Höheweg 37, Interlaken",46.6861,7.8564,46.6861,7.8564
22038,"{'location': {'lat': 46.989853, 'lng': 6.92220...",https://maps.gstatic.com/mapfiles/place_api/ic...,d381f201f5cfb26666faf643bd58eb4e597195ca,Semaphore,NaN,"[{'height': 430, 'html_attributions': ['<a hre...",ChIJXcHqIBsKjkcRDA5DG3OJ85A,"{'compound_code': 'XWQC+WV Neuchâtel, Switzerl...",nan,nan,ChIJXcHqIBsKjkcRDA5DG3OJ85A,GOOGLE,"[art_gallery, point_of_interest, establishment]",nan,"Rue de l'Evole 35A, Neuchâtel",46.9899,6.9222,46.9899,6.9222
81607,"{'location': {'lat': 46.5883256, 'lng': 7.0843...",https://maps.gstatic.com/mapfiles/place_api/ic...,3d2185f03df4c6a19d6a312e513b70dd67126dce,La Croix Blanche Epagny,{'open_now': False},"[{'height': 364, 'html_attributions': ['<a hre...",ChIJewlmjpaJjkcR2NdhcrBSJiI,"{'compound_code': 'H3QM+8P Gruyères, Switzerla...",2.0000,4.3000,ChIJewlmjpaJjkcR2NdhcrBSJiI,GOOGLE,"[restaurant, food, point_of_interest, establis...",366.0000,"Route de l'Intyamon 336, Epagny",46.5883,7.0844,46.5883,7.0844
92291,"{'location': {'lat': 46.0971395, 'lng': 7.2248...",https://maps.gstatic.com/mapfiles/place_api/ic...,f9bfa88eb0c1d16d1a64083f6b07f63a96931760,Coiffure les Cytises,NaN,NaN,ChIJz9SdLUvOjkcRvHVLqUM0Plc,"{'compound_code': '36WF+VW Bagnes, Switzerland...",nan,4.0000,ChIJz9SdLUvOjkcRvHVLqUM0Plc,GOOGLE,"[hair_care, point_of_interest, establishment]",1.0000,"Rue du Centre sportif 1, Bagnes",46.0971,7.2248,46.0971,7.2248


In [3]:
print()
print("Around {0:.2f}% of Hot OSM points are considered valid".format(len(valid_pois) / len(pois)))


Around 0.58% of Hot OSM points are considered valid


### Please check <font color="red"> Valid POIs Analysis - Google Places.html</font> for column values frequency

## POIs distribution

In [4]:
p = None

n_sample = 10000

plot_pois = valid_pois.sample(n_sample)

p = plot.plot_poi(plot_pois, title="", figure=p, width=600, height=400, lat_col="latitude", lon_col="longitude")
    
show(p)
print("Sample size: {}".format(n_sample))

Sample size: 10000


## Computing KNN POIs for each Stop Region

In [5]:
def knn_by_clusters_google_places(centroids, pois, k_neighbors, k_partitions=4):
    centroids["cluster"] = partitions(centroids, k_partitions=k_partitions, columns=["latitude", "longitude"])
    
    user_knn_pois = []

    for partition in centroids["cluster"].drop_duplicates():
        print("--")
        print("Partition: {}".format(partition))
        partition_centroids = centroids[centroids["cluster"] == partition]
        print("Stop Regions in this parittion: {}".format(len(partition_centroids)))
        
        pois["latitude"] = pois["lat_4326"]
        pois["longitude"] = pois["lon_4326"]

        close_pois = geo.grab_pois_by_stop_region_bounding_box_expand_fixed(pois, partition_centroids, expand_value=0.004)
        
        if len(close_pois) == 0:
            continue
        
        user_knn_pois = user_knn_pois + geo.knn_pois_google_places(partition_centroids, close_pois, k=k_neighbors)
        
    return user_knn_pois


In [6]:
users = csv_dao.list_stop_region_usernames()

In [7]:
k_neighbors = 30
k_partitions = 6

for user in users:
    print("\n\n")
    print("**********")
    print("User:", user)
    
    sr_centroids = csv_dao.load_user_stop_regions_centroids(user)
    
    print("Stop Regions:", len(sr_centroids))
    print("-----")
    
    user_dir = os.getcwd() + "/outputs/google_places_sr_knn/" + str(user)
    try:
        os.mkdir(user_dir)
    except FileExistsError:
        print("User already computed... skipping")
        continue
        #shutil.rmtree(user_dir)
        #os.mkdir(user_dir)
    except FileNotFoundError:
        os.mkdir(user_dir)
        
    print("Computing")
    
    if len(sr_centroids) < k_partitions:
        user_knn_pois = knn_by_clusters_google_places(sr_centroids, valid_pois, k_neighbors, k_partitions=1)
    
    else:
        user_knn_pois = knn_by_clusters_google_places(sr_centroids, valid_pois, k_neighbors, k_partitions=k_partitions)
    
    for knn in user_knn_pois:
        sr_id = knn["sr_id"].drop_duplicates().item()
        knn.to_csv(user_dir + "/" + "/" + "sr_" + sr_id + "_knn" + ".csv", index=False)




**********
User: 6189
Stop Regions: 574
-----
User already computed... skipping



**********
User: 5936
Stop Regions: 761
-----
User already computed... skipping



**********
User: 6087
Stop Regions: 610
-----
User already computed... skipping



**********
User: 5973
Stop Regions: 6079
-----
User already computed... skipping



**********
User: 6085
Stop Regions: 1398
-----
User already computed... skipping



**********
User: 6074
Stop Regions: 45
-----
User already computed... skipping



**********
User: 6012
Stop Regions: 517
-----
User already computed... skipping



**********
User: 5982
Stop Regions: 104
-----
User already computed... skipping



**********
User: 5948
Stop Regions: 996
-----
User already computed... skipping



**********
User: 5974
Stop Regions: 1038
-----
User already computed... skipping



**********
User: 6090
Stop Regions: 85
-----
User already computed... skipping



**********
User: 6199
Stop Regions: 240
-----
User already computed... skipping





--
Partition: 5
Stop Regions in this parittion: 6
1455 pois out of 76743
--
Partition: 2
Stop Regions in this parittion: 31
7210 pois out of 76743
--
Partition: 4
Stop Regions in this parittion: 14
194 pois out of 76743
--
Partition: 3
Stop Regions in this parittion: 21
6947 pois out of 76743
--
Partition: 1
Stop Regions in this parittion: 5
535 pois out of 76743



**********
User: 6086
Stop Regions: 450
-----
Computing
--
Partition: 1
Stop Regions in this parittion: 400
17204 pois out of 76743
--
Partition: 4
Stop Regions in this parittion: 17
8108 pois out of 76743
--
Partition: 2
Stop Regions in this parittion: 6
367 pois out of 76743
--
Partition: 3
Stop Regions in this parittion: 12
672 pois out of 76743
--
Partition: 0
Stop Regions in this parittion: 8
49 pois out of 76743
--
Partition: 5
Stop Regions in this parittion: 7
342 pois out of 76743



**********
User: 6100
Stop Regions: 6
-----
Computing
--
Partition: 4
Stop Regions in this parittion: 1
568 pois out of 76743
--
Parti

Stop Regions: 345
-----
User already computed... skipping



**********
User: 6058
Stop Regions: 661
-----
User already computed... skipping



**********
User: 6001
Stop Regions: 1686
-----
User already computed... skipping



**********
User: 5964
Stop Regions: 1057
-----
User already computed... skipping



**********
User: 5950
Stop Regions: 649
-----
User already computed... skipping



**********
User: 5991
Stop Regions: 1849
-----
User already computed... skipping



**********
User: 6177
Stop Regions: 3125
-----
User already computed... skipping



**********
User: 5961
Stop Regions: 2161
-----
User already computed... skipping



**********
User: 6176
Stop Regions: 910
-----
User already computed... skipping



**********
User: 6007
Stop Regions: 102
-----
User already computed... skipping



**********
User: 5956
Stop Regions: 308
-----
User already computed... skipping



**********
User: 6182
Stop Regions: 6
-----
User already computed... skipping



**********
User: 6171
S

Stop Regions: 725
-----
User already computed... skipping
